In [2]:
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 9.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
from rdflib import Graph, RDFS, URIRef
from rdflib.namespace import RDF, OWL
from collections import defaultdict

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd

chunks = pd.read_csv(
    "/content/drive/MyDrive/casereports/top_class_code_entries.csv",
    on_bad_lines='skip',
    low_memory=False,
    chunksize=50000  # Adjust down if it still crashes
)

nif_df = pd.concat(chunks, ignore_index=True)


In [6]:
for col in ['Preferred Label', 'label', 'Synonyms', 'Definitions', 'domain']:
    if col in nif_df.columns:
        nif_df[col] = nif_df[col].fillna('').astype(str).str.lower()


In [ ]:
import pandas as pd
from collections import defaultdict


# Ensure relevant columns exist and convert to lowercase strings
for col in ['domain', 'label', 'Synonyms', 'Definitions']:
    if col in nif_df.columns:
        nif_df[col] = nif_df[col].fillna('').astype(str).str.lower()

# Initialize empty dictionary
domain_terms = defaultdict(set)

# Loop through rows
for _, row in nif_df.iterrows():
    domain = row['domain'].strip().lower()
    if not domain:
        continue

    for col in ['label', 'Synonyms', 'Definitions']:
        if col in row and row[col]:
            terms = row[col].split('|') if '|' in row[col] else [row[col]]
            for term in terms:
                cleaned = term.strip()
                if cleaned and len(cleaned) > 2:
                    domain_terms[domain].add(cleaned)

# Convert sets to sorted lists
domain_terms = {k: sorted(list(v)) for k, v in domain_terms.items()}

# Save as JSON
import json
with open("/content/drive/MyDrive/casereports/domain_terms_from_csv.json", "w") as f:
    json.dump(domain_terms, f, indent=2)

print("✅ Extracted domain-specific terms from CSV.")


✅ Extracted domain-specific terms from CSV.


In [ ]:
for domain, terms in domain_terms.items():
    print(f"\nDomain: {domain} ({len(terms)} terms)")
    print(", ".join(terms[:10]))  # Show only the first 10 terms



Domain: gene (219228 terms)
"a primer containing an snv at the 3' end for accurate genotyping., "this tripartite complex named par6/par3/apkc is conserved from worms to vertebrates" (pmid:18005931). in yeast, the components are bem1, cdc24, and cla4., 'de novo' actin filament nucleation, 'de novo' amp biosynthetic process, 'de novo' co-translational protein folding, 'de novo' cotranslational protein folding, 'de novo' ctp biosynthetic process, 'de novo' cytidine 5'-triphosphate biosynthetic process, 'de novo' gdp-l-fucose anabolism, 'de novo' gdp-l-fucose biosynthesis

Domain: taxonomy (64210 terms)
"acetobacter melanogenum (sic) var. malto-saccharovorans" frateur 1950, "acetobacter melanogenum (sic) var. maltovorans" frateur 1950, "acetobacterium" ludwig 1898, "achromobacter globiformis" (conn 1928) bergey et al. 1930, "aciduliprofundales", "actinobacillus muris" (de mello and pais 1918) wilson and miles 1955, "aerobacter" hormaeche and edwards 1958, "alphaproteobacteriota" whitman e

In [1]:
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 11.2 MB/s eta 0:00:00


In [4]:
import pandas as pd
from rdflib import Graph, URIRef
from rdflib.namespace import RDFS
from collections import defaultdict
import json

# Load class-to-domain mapping
chunks = pd.read_csv(
    "/content/drive/MyDrive/casereports/top_class_code_entries.csv",
    on_bad_lines='skip',
    low_memory=False,
    chunksize=50000  # Adjust down if it still crashes
)

df = pd.concat(chunks, ignore_index=True)

df = df[['Class ID', 'domain']].dropna()
df['Class ID'] = df['Class ID'].astype(str).str.strip()
df['domain'] = df['domain'].astype(str).str.strip().str.lower()

# Build lookup: {Class ID URI → domain}
id_to_domain = dict(zip(df['Class ID'], df['domain']))

# Parse OWL ontology file
g = Graph()
g.parse("/content/drive/MyDrive/casereports/owlapi.xrdf", format="xml")

# Autodetect all namespaces
print("Registered Namespaces:")
for prefix, ns in g.namespaces():
    print(f"  {prefix}: {ns}")

# Store extracted terms by domain
domain_terms = defaultdict(set)

# Loop over each unique Class ID
for class_uri_str, domain in id_to_domain.items():
    uri = URIRef(class_uri_str)

    # --- Extract rdfs:label
    label = g.value(subject=uri, predicate=RDFS.label)
    if label:
        domain_terms[domain].add(str(label).lower())

    # --- Extract all other string properties (like synonyms or definitions)
    for pred, obj in g.predicate_objects(subject=uri):
        if isinstance(obj, str) and len(obj.strip()) > 2:
            domain_terms[domain].add(obj.strip().lower())

# Convert sets to sorted lists
domain_terms = {dom: sorted(terms) for dom, terms in domain_terms.items()}

# Save results
with open("/content/drive/MyDrive/casereports/domain_terms_from_xrdf.json", "w") as f:
    json.dump(domain_terms, f, indent=2)

print(f"\nExtracted terms for {len(domain_terms)} domains.")


Registered Namespaces:
  brick: https://brickschema.org/schema/Brick#
  csvw: http://www.w3.org/ns/csvw#
  dc: http://purl.org/dc/elements/1.1/
  dcat: http://www.w3.org/ns/dcat#
  dcmitype: http://purl.org/dc/dcmitype/
  dcterms: http://purl.org/dc/terms/
  dcam: http://purl.org/dc/dcam/
  doap: http://usefulinc.com/ns/doap#
  foaf: http://xmlns.com/foaf/0.1/
  geo: http://www.opengis.net/ont/geosparql#
  odrl: http://www.w3.org/ns/odrl/2/
  org: http://www.w3.org/ns/org#
  prof: http://www.w3.org/ns/dx/prof/
  prov: http://www.w3.org/ns/prov#
  qb: http://purl.org/linked-data/cube#
  schema: https://schema.org/
  sh: http://www.w3.org/ns/shacl#
  skos: http://www.w3.org/2004/02/skos/core#
  sosa: http://www.w3.org/ns/sosa/
  ssn: http://www.w3.org/ns/ssn/
  time: http://www.w3.org/2006/time#
  vann: http://purl.org/vocab/vann/
  void: http://rdfs.org/ns/void#
  wgs: https://www.w3.org/2003/01/geo/wgs84_pos#
  owl: http://www.w3.org/2002/07/owl#
  rdf: http://www.w3.org/1999/02/22-rdf

In [5]:
chunks = pd.read_csv(
    "/content/drive/MyDrive/casereports/top_class_code_entries.csv",
    on_bad_lines='skip',
    low_memory=False,
    chunksize=50000  # Adjust down if it still crashes
)

df = pd.concat(chunks, ignore_index=True)
df['domain'].nunique()

10

In [7]:
for domain, terms in domain_terms.items():
    print(f"\nDomain: {domain} ({len(terms)} terms)")
    print(", ".join(terms[:10]))  # Show only the first 10 terms


Domain: gene (373503 terms)
"a primer containing an snv at the 3' end for accurate genotyping., "this tripartite complex named par6/par3/apkc is conserved from worms to vertebrates" (pmid:18005931). in yeast, the components are bem1, cdc24, and cla4., #25320, 'de novo' actin filament nucleation, 'de novo' amp biosynthetic process, 'de novo' co-translational protein folding, 'de novo' cotranslational protein folding, 'de novo' ctp biosynthetic process, 'de novo' cytidine 5'-triphosphate biosynthetic process, 'de novo' gdp-l-fucose anabolism

Domain: taxonomy (123167 terms)
"acetobacter melanogenum (sic) var. malto-saccharovorans" frateur 1950, "acetobacter melanogenum (sic) var. maltovorans" frateur 1950, "acetobacterium" ludwig 1898, "achromobacter globiformis" (conn 1928) bergey et al. 1930, "aciduliprofundales", "actinobacillus muris" (de mello and pais 1918) wilson and miles 1955, "aerobacter" hormaeche and edwards 1958, "alphaproteobacteriota" whitman et al. 2018, "autotrophic rho

**Cleaning extracted keywords**

In [6]:
!pip install nltk

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [26]:
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

# Domain-specific noise terms
domain_noise_terms = {
    "et al", "figure", "fig", "table", "www", "http", "https",
    "pmid", "doi", "data", "method", "result", "conclusion",
    "introduction", "study", "research", "effect", "model",
    "findings", "values", "sample"
}

# Date patterns
date_patterns = [
    r'\b\d{1,2}-[a-z]{3}-\d{2}\b',               # 15-may-10
    r'\b\d{4}-\d{2}-\d{2}( \d{6})?\b',           # 2009-07-10 or 2009-07-10 140630
    r'\b\d{2}/\d{2}/\d{4}\b',                    # 10/12/2020
    r'\b\d{4}/\d{2}/\d{2}\b',                    # 2020/12/10
    r'\b\d{4}-\d{2}-\d{2}t\d{6}z?\b',            # 2008-12-18t124919z or 2008-12-18t124919
]


def is_date_like(term):
    for pattern in date_patterns:
        if re.search(pattern, term):
            return True
    return False

def is_ontology_id_like(term):
    return re.fullmatch(r'\d{5,7}', term) is not None

# Detect if a term looks like a bad measurement or random alphanumeric junk
def is_garbage_term(term):
    return bool(re.fullmatch(r'\d+\s+\d+\s*(cm|mm|nm)?', term)) or bool(re.fullmatch(r'1\s+[a-zA-Z0-9]+', term))

def is_junk_measurement(term):
    # Matches patterns like '10 a2', '10 b4', '1 d1'
    if re.fullmatch(r'\d+\s+[a-zA-Z]+\d*', term):
        return True
    # Matches mixed units like '10 feet 3 ml'
    if re.search(r'\b(cm|mm|feet|inch|ml|g|mg|l|litre)\b.*\b(cm|mm|feet|inch|ml|g|mg|l|litre)\b', term):
        return True
    return False


def clean_keywords(keyword_list):
    cleaned = set()
    for kw in keyword_list:
        kw = kw.lower().strip('"\'')
        kw = re.sub(r"[^\w\s\-\+\(\)']", '', kw)
        kw = re.sub(r'\s+', ' ', kw).strip()

        # Skip if it's short, a stopword, noise term, or date-like
        if (
            len(kw) > 2 and
            kw not in stop_words and
            kw not in domain_noise_terms and
            not is_date_like(kw) and
            not is_ontology_id_like(kw) and
            not is_garbage_term(kw) and
            not is_junk_measurement(kw)
           ):
            cleaned.add(kw)
    return sorted(cleaned)


# Load your domain-term JSON
with open("/content/drive/MyDrive/casereports/domain_terms_from_csv.json", "r") as f:
    domain_terms = json.load(f)

# Clean all domains
cleaned_domain_terms = {
    domain: clean_keywords(terms)
    for domain, terms in domain_terms.items()
}

# Save cleaned terms to JSON
with open("/content/drive/MyDrive/casereports/cleaned_domain_terms.json", "w") as f:
    json.dump(cleaned_domain_terms, f, indent=2)

print("✅ Cleaned and saved domain-specific keywords.")


✅ Cleaned and saved domain-specific keywords.


In [27]:
for domain, terms in cleaned_domain_terms.items():
    print(f"\nDomain: {domain} ({len(terms)} terms)")
    print(", ".join(terms[:10]))  # Show only the first 10 terms


Domain: gene (217469 terms)
(+)-(10r)-germacrene a synthase activity, (+)-2-epi-prezizaene synthase activity, (+)-3'-hydroxylarreatricin anabolism, (+)-3'-hydroxylarreatricin biosynthesis, (+)-3'-hydroxylarreatricin biosynthetic process, (+)-3'-hydroxylarreatricin formation, (+)-3'-hydroxylarreatricin synthesis, (+)-aba 8'-hydroxylase activity, (+)-abscisic acid 8'-hydroxylase activity, (+)-abscisic acid d-glucopyranosyl ester transmembrane transport

Domain: taxonomy (64040 terms)
(antopocerus), (dorsilopha), (engiscaptomyza), (ralvrhdvast891989sp), (ralvrhdvbs891989it), (ralvrhdvgh1989ge), (ralvrhdvrcv1995it), (ralvrhdvsd1989fr), (ralvrhdvv3511991ck), 16sri (aster yellows group)

Domain: brain_atlas (7091 terms)
(cocomac), (cocomac) accessor basal nucleus (amygdala) ventromedial division, (cocomac) accessory basal (magnocellular) nucleus of the amygdala, (cocomac) accessory basal (parvocellular) nucleus of the amygdala, (cocomac) accessory basal amygdaloid nucleus, (cocomac) accesso

In [24]:
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

# Domain-specific noise terms
domain_noise_terms = {
    "et al", "figure", "fig", "table", "www", "http", "https",
    "pmid", "doi", "data", "method", "result", "conclusion",
    "introduction", "study", "research", "effect", "model",
    "findings", "values", "sample"
}

# Date patterns
date_patterns = [
    r'\b\d{1,2}-[a-z]{3}-\d{2}\b',               # 15-may-10
    r'\b\d{4}-\d{2}-\d{2}( \d{6})?\b',           # 2009-07-10 or 2009-07-10 140630
    r'\b\d{2}/\d{2}/\d{4}\b',                    # 10/12/2020
    r'\b\d{4}/\d{2}/\d{2}\b',                    # 2020/12/10
    r'\b\d{4}-\d{2}-\d{2}t\d{6}z?\b',            # 2008-12-18t124919z or 2008-12-18t124919
]


def is_date_like(term):
    for pattern in date_patterns:
        if re.search(pattern, term):
            return True
    return False

def is_ontology_id_like(term):
    return re.fullmatch(r'\d{5,7}', term) is not None

# Detect if a term looks like a bad measurement or random alphanumeric junk
def is_garbage_term(term):
    return bool(re.fullmatch(r'\d+\s+\d+\s*(cm|mm|nm)?', term)) or bool(re.fullmatch(r'1\s+[a-zA-Z0-9]+', term))

def is_junk_measurement(term):
    # Matches patterns like '10 a2', '10 b4', '1 d1'
    if re.fullmatch(r'\d+\s+[a-zA-Z]+\d*', term):
        return True
    # Matches mixed units like '10 feet 3 ml'
    if re.search(r'\b(cm|mm|feet|inch|ml|g|mg|l|litre)\b.*\b(cm|mm|feet|inch|ml|g|mg|l|litre)\b', term):
        return True
    return False


def clean_keywords(keyword_list):
    cleaned = set()
    for kw in keyword_list:
        kw = kw.lower().strip('"\'')
        kw = re.sub(r"[^\w\s\-\+\(\)']", '', kw)
        kw = re.sub(r'\s+', ' ', kw).strip()

        # Skip if it's short, a stopword, noise term, or date-like
        if (
            len(kw) > 2 and
            kw not in stop_words and
            kw not in domain_noise_terms and
            not is_date_like(kw) and
            not is_ontology_id_like(kw) and
            not is_garbage_term(kw) and
            not is_junk_measurement(kw)
           ):
            cleaned.add(kw)
    return sorted(cleaned)


# Load your domain-term JSON
with open("/content/drive/MyDrive/casereports/domain_terms_from_xrdf.json", "r") as f:
    domain_terms = json.load(f)

# Clean all domains
cleaned_domain_terms = {
    domain: clean_keywords(terms)
    for domain, terms in domain_terms.items()
}

# Save cleaned terms to JSON
with open("/content/drive/MyDrive/casereports/cleaned_domain_terms_xrdf.json", "w") as f:
    json.dump(cleaned_domain_terms, f, indent=2)

print("✅ Cleaned and saved domain-specific keywords.")


✅ Cleaned and saved domain-specific keywords.


In [25]:
for domain, terms in cleaned_domain_terms.items():
    print(f"\nDomain: {domain} ({len(terms)} terms)")
    print(", ".join(terms[:10]))  # Show only the first 10 terms


Domain: gene (354200 terms)
(+)-(10r)-germacrene a synthase activity, (+)-2-epi-prezizaene synthase activity, (+)-3'-hydroxylarreatricin anabolism, (+)-3'-hydroxylarreatricin biosynthesis, (+)-3'-hydroxylarreatricin biosynthetic process, (+)-3'-hydroxylarreatricin formation, (+)-3'-hydroxylarreatricin synthesis, (+)-aba 8'-hydroxylase activity, (+)-abscisic acid 8'-hydroxylase activity, (+)-abscisic acid d-glucopyranosyl ester transmembrane transport

Domain: taxonomy (122996 terms)
(antopocerus), (dorsilopha), (engiscaptomyza), (ralvrhdvast891989sp), (ralvrhdvbs891989it), (ralvrhdvgh1989ge), (ralvrhdvrcv1995it), (ralvrhdvsd1989fr), (ralvrhdvv3511991ck), 16sri (aster yellows group)

Domain: brain_atlas (18199 terms)
(cocomac), (cocomac) accessor basal nucleus (amygdala) ventromedial division, (cocomac) accessory basal (magnocellular) nucleus of the amygdala, (cocomac) accessory basal (parvocellular) nucleus of the amygdala, (cocomac) accessory basal amygdaloid nucleus, (cocomac) acces

In [28]:
import json
from collections import defaultdict

# Load both JSON files
with open('/content/drive/MyDrive/casereports/cleaned_domain_terms.json', 'r') as f1:
    csv_terms = json.load(f1)

with open('/content/drive/MyDrive/casereports/cleaned_domain_terms_xrdf.json', 'r') as f2:
    xrdf_terms = json.load(f2)

# Initialize a combined dictionary
combined_terms = defaultdict(set)

# Add terms from CSV
for domain, terms in csv_terms.items():
    for term in terms:
        combined_terms[domain].add(term.strip().lower())

# Add terms from XRDF
for domain, terms in xrdf_terms.items():
    for term in terms:
        combined_terms[domain].add(term.strip().lower())

# Convert sets to sorted lists
combined_terms = {k: sorted(list(v)) for k, v in combined_terms.items()}

# Save to new JSON file
with open('/content/drive/MyDrive/casereports/combined_domain_keywords.json', 'w') as f:
    json.dump(combined_terms, f, indent=2)

print("✅ Successfully combined and saved domain-specific terms.")


✅ Successfully combined and saved domain-specific terms.


In [29]:
for domain, terms in combined_terms.items():
    print(f"\nDomain: {domain} ({len(terms)} terms)")
    print(", ".join(terms[:10]))  # Show only the first 10 terms


Domain: gene (354217 terms)
(+)-(10r)-germacrene a synthase activity, (+)-2-epi-prezizaene synthase activity, (+)-3'-hydroxylarreatricin anabolism, (+)-3'-hydroxylarreatricin biosynthesis, (+)-3'-hydroxylarreatricin biosynthetic process, (+)-3'-hydroxylarreatricin formation, (+)-3'-hydroxylarreatricin synthesis, (+)-aba 8'-hydroxylase activity, (+)-abscisic acid 8'-hydroxylase activity, (+)-abscisic acid d-glucopyranosyl ester transmembrane transport

Domain: taxonomy (122996 terms)
(antopocerus), (dorsilopha), (engiscaptomyza), (ralvrhdvast891989sp), (ralvrhdvbs891989it), (ralvrhdvgh1989ge), (ralvrhdvrcv1995it), (ralvrhdvsd1989fr), (ralvrhdvv3511991ck), 16sri (aster yellows group)

Domain: brain_atlas (18199 terms)
(cocomac), (cocomac) accessor basal nucleus (amygdala) ventromedial division, (cocomac) accessory basal (magnocellular) nucleus of the amygdala, (cocomac) accessory basal (parvocellular) nucleus of the amygdala, (cocomac) accessory basal amygdaloid nucleus, (cocomac) acces